In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
## Load the trained model, scaler pickle, one hot
model = load_model('model.h5')

##load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb')as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb')as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb')as file:
    scaler = pickle.load(file)

In [3]:
#Example Input data
#we need to convert text into numerical are geography and gender
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:
#One hot encoding geography
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
#AttributeError: 'dict' object has no attribute 'reset_index' to resolve this error we need to convert inptu data from key value pair to dataframe
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
##Encode Categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:
# Concatination one hot encoded
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
# Add missing columns from the original dataset
original_columns = ['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Gender', 'Age', 'Tenure', 
                    'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'] + \
                   list(label_encoder_geo.get_feature_names_out(['Geography']))

for col in original_columns:
    if col not in input_df.columns:
        input_df[col] = 0  # Add placeholder values for missing columns

# Reorder columns to match the scaler's fit order
input_df = input_df[original_columns]

# Scale the input data
input_scaled = scaler.transform(input_df)

print(input_scaled)

[[-1.73595179e+00 -2.18402312e+02 -1.77330994e+00 -5.35985161e-01
   9.13247552e-01  1.04793593e-01 -6.95393486e-01 -2.57811188e-01
   8.08436154e-01  6.49202671e-01  9.74816989e-01 -8.76832208e-01
   1.00150113e+00 -5.79467227e-01 -5.76388018e-01]]


In [12]:
##Predict chrun
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step


array([[7.018667e-13]], dtype=float32)

In [13]:
##probability of prediction
prediction_proba = prediction[0][0]

In [14]:
prediction_proba

7.018667e-13

In [ ]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
